<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#집계-연산" data-toc-modified-id="집계-연산-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>집계 연산</a></span><ul class="toc-item"><li><span><a href="#집계-함수" data-toc-modified-id="집계-함수-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>집계 함수</a></span><ul class="toc-item"><li><span><a href="#count" data-toc-modified-id="count-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>count</a></span></li><li><span><a href="#countDistinct" data-toc-modified-id="countDistinct-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>countDistinct</a></span></li><li><span><a href="#approx_count_distinct" data-toc-modified-id="approx_count_distinct-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>approx_count_distinct</a></span></li><li><span><a href="#first,-last" data-toc-modified-id="first,-last-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>first, last</a></span></li><li><span><a href="#min,-max" data-toc-modified-id="min,-max-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>min, max</a></span></li><li><span><a href="#sum" data-toc-modified-id="sum-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>sum</a></span></li><li><span><a href="#sumDistinct" data-toc-modified-id="sumDistinct-1.1.7"><span class="toc-item-num">1.1.7&nbsp;&nbsp;</span>sumDistinct</a></span></li><li><span><a href="#avg,-mean" data-toc-modified-id="avg,-mean-1.1.8"><span class="toc-item-num">1.1.8&nbsp;&nbsp;</span>avg, mean</a></span></li><li><span><a href="#분산과-표준편차" data-toc-modified-id="분산과-표준편차-1.1.9"><span class="toc-item-num">1.1.9&nbsp;&nbsp;</span>분산과 표준편차</a></span></li><li><span><a href="#비대칭도(skewness),-첨도(kurtosis)" data-toc-modified-id="비대칭도(skewness),-첨도(kurtosis)-1.1.10"><span class="toc-item-num">1.1.10&nbsp;&nbsp;</span>비대칭도(skewness), 첨도(kurtosis)</a></span></li><li><span><a href="#공분산과-상관관계" data-toc-modified-id="공분산과-상관관계-1.1.11"><span class="toc-item-num">1.1.11&nbsp;&nbsp;</span>공분산과 상관관계</a></span></li><li><span><a href="#복합-데이터-타입의-집계" data-toc-modified-id="복합-데이터-타입의-집계-1.1.12"><span class="toc-item-num">1.1.12&nbsp;&nbsp;</span>복합 데이터 타입의 집계</a></span></li></ul></li><li><span><a href="#그룹화" data-toc-modified-id="그룹화-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>그룹화</a></span><ul class="toc-item"><li><span><a href="#표현식을-이용한-그룹화" data-toc-modified-id="표현식을-이용한-그룹화-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>표현식을 이용한 그룹화</a></span></li></ul></li><li><span><a href="#윈도우-함수" data-toc-modified-id="윈도우-함수-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>윈도우 함수</a></span></li></ul></li></ul></div>

# 집계 연산
- 집계 시 인지해야 할 중요한 점은
    - 집계한다는 것은 빅데이터에서 정확한 답을 얻기 위함임. 그런데 어느정도 즉, 수용가능한 정도의 정확도에 맞춰 근사치를 계산하는 것이 연산, 네트워크, 저장소 등의 비용을 고려하는 것임
    - 따라서 **근사치 계산용 함수**를 사용해 스파크 잡의 실행과 속도를 개선할 수 있음

In [1]:
df = spark.read.format('csv')\
          .option('header', 'true')\
          .option('inferSchema', 'true')\
          .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/retail-data/all/*.csv')\
          .coalesce(5)  # 컬레스를 사용해 리파티셔닝(파티션 감소)

# 해당 데이터프레임을 메모리에 캐싱
df.cache()
# 트랜스포메이션 결과(액션) 관찰하기 위해 임시 테이블로 생성
df.createOrReplaceTempView('dfTable')

- ``count()``메소드는 즉시 수행하는 액션 메소드임! 
    - 데이터프레임의 행 개수를 집계하는 데 사용되기도 하지만 메모리에 데이터프레임 캐싱작업을 수행하는 용도로도 사용됨!

In [2]:
count = df.count()
print(count)
print(count == 541909)

541909
True


## 집계 함수

### count
- 해당 예제에서는 액션이 아닌 **트랜스포메이션(lazy execution)** 으로 동작함
- 두 가지 방식으로 사용 가능
    * ``count("column_name")``
        - 특정 칼럼명을 지정하면 ``null``값 제외하고 집계
    * ``count(*)``
        - 단, SQL과 마착나지로 ``*`` 사용해서 카운트 시, 만약 칼럼들에 ``null``값이 포함되어 있다면 ``null``이 있는 것도 모두 집계
        

In [3]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import count 

df.select(count('StockCode').alias('count')).show()

+------+
| count|
+------+
|541909|
+------+



In [5]:
df.select(count('*')).show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



### countDistinct

- unique한 row 개수를 출력하는데, 이 메소드는 주로 개별 컬럼에다가 적용하는것이 적합

In [6]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct('CustomerID').alias('Customer_cnt')).show()

+------------+
|Customer_cnt|
+------------+
|        4372|
+------------+



### approx_count_distinct
- 대규모 데이터셋을 다루다 보면 정확한 unique한 개수는 필요없고 대략적인 근사치만 필요할 때가 있음
- 추가 파라미터인 최대 추정 오류율(maximum estimation error)이 존재하는데, 0.1이면 오류율 10%을 의미
    - 단, 39%의 오류율까지만 허용함. 그 이상 입력 시 에러 발생
- ``countDistinct``보다 훨씬 속도가 빠르며, 데이터셋이 매우 커짐에 따라 해당 메서드의 유용성이 증가(단, 대략적인 근사치가 필요할 때만!)

In [7]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct('CustomerID', 0.1).alias('approx_Customer_cnt')).show()

+-------------------+
|approx_Customer_cnt|
+-------------------+
|               4336|
+-------------------+



In [8]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct('CustomerID', 0.39).alias('approx_Customer_cnt')).show()

+-------------------+
|approx_Customer_cnt|
+-------------------+
|               5955|
+-------------------+



### first, last
- 데이터프레임 특정 칼럼의 첫 번째 값 또는 마지막 값을 반환
- 이 함수들은 Row-based으로 동작함
    - Row-based/Column-based 설명 <a href='https://bi-insider.com/business-intelligence/column-and-row-based-database-storage/'>링크</a>

In [9]:
from pyspark.sql.functions import first, last

df.select(first('Description'), last('Description')).show(truncate=False)

+----------------------------------+-----------------------------+
|first(Description)                |last(Description)            |
+----------------------------------+-----------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|BAKING SET 9 PIECE RETROSPOT |
+----------------------------------+-----------------------------+



### min, max
- 데이터프레임 특정 칼럼의 최솟값, 최대값을 반환(문자열도 오름,내림차순으로 자동 정렬 후 추출됨)

In [10]:
from pyspark.sql.functions import max, min 

df.select(max('Country'), min('Country'), max('UnitPrice'), min('UnitPrice')).show()

+------------+------------+--------------+--------------+
|max(Country)|min(Country)|max(UnitPrice)|min(UnitPrice)|
+------------+------------+--------------+--------------+
| Unspecified|   Australia|       38970.0|     -11062.06|
+------------+------------+--------------+--------------+



### sum
- 모든 값을 합산
- 문자열을 sum 하면 null로 반환 됨

In [11]:
from pyspark.sql.functions import sum

df.select(sum('UnitPrice')).show()

+----------------+
|  sum(UnitPrice)|
+----------------+
|2498803.97400038|
+----------------+



In [12]:
# 문자열을 sum 하면 null로 반환 됨
df.select(sum('Country')).show()

+------------+
|sum(Country)|
+------------+
|        null|
+------------+



### sumDistinct
- unique한 값들만 합산 가능

In [13]:
from pyspark.sql.functions import sumDistinct

df.select(sumDistinct('UnitPrice')).show()

+-----------------------+
|sum(DISTINCT UnitPrice)|
+-----------------------+
|      611388.3910000001|
+-----------------------+



### avg, mean
- ``sum / count``로 평균값을 직접 구할 수 있지만, avg, mean으로 대체 가능
    - ``avg``는 ``pyspark.sql.functions`` 패키지에 있지만,
    - ``mean``은 ``expr`` 메소드를 사용한 후 쿼리문 형태로 사용 가능함
- 참고로 대부분의 집계 함수는 unique한 값들만 사용해 집계를 수행하는 방식을 지원함!

In [14]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(count('Quantity').alias('total_transactions'),
         sum('Quantity').alias('total_purchases'),
         avg('Quantity').alias('avg_quantity'),
         expr('mean(Quantity)').alias('mean_quantity')) \
  .selectExpr('total_purchases / total_transactions',
             'avg_quantity',
             'mean_quantity').show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|    avg_quantity|   mean_quantity|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



### 분산과 표준편차
- 스파크는 표본분산(표준편차)와 모분산(표준편차)를 개별로 제공
    - 위 2개 계산 방식은 다르기 떄문에 메서드도 다름


In [15]:
from pyspark.sql.functions import var_samp, stddev_samp # sample = 표본
from pyspark.sql.functions import var_pop, stddev_pop   # pop=population = 모집단

df.select(var_samp('Quantity'), var_pop('Quantity'), stddev_samp('Quantity'), stddev_pop('Quantity')).show()

+------------------+------------------+---------------------+--------------------+
|var_samp(Quantity)| var_pop(Quantity)|stddev_samp(Quantity)|stddev_pop(Quantity)|
+------------------+------------------+---------------------+--------------------+
|47559.391409298754|47559.303646609056|   218.08115785023418|  218.08095663447796|
+------------------+------------------+---------------------+--------------------+



### 비대칭도(skewness), 첨도(kurtosis)
- 확률변수, 확률분포를 기반으로 데이터를 모델링할 때 해당 척도들을 알아보는 것이 중요할 때 있음
- 값을 모두 정규화시킨 후, 왜도, 첨도를 계산했을 때,
    - 왜도
        - 양수: 왼쪽으로 기울어짐(오른쪽으로 꼬리가 긴)
        - 음수: 오른쪽으로 기울어짐(왼쪽으로 꼬리가 긴)
        - 0: 정규분포 모양과 유사
    - 첨도
        - 양수: 뾰족함(즉, 표준편차값이 작을 때 모양)
        - 음수: 퍼져있음(즉, 표준편차값이 클 때 모양)
        - 0: 정규분포 모양과 유사

In [16]:
from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("UnitPrice"), kurtosis("UnitPrice")).show()

+-------------------+-------------------+
|skewness(UnitPrice)|kurtosis(UnitPrice)|
+-------------------+-------------------+
| 186.50645547025985| 59005.174662675585|
+-------------------+-------------------+



### 공분산과 상관관계
- 공분산은 데이터 입력의 범위를 고려하지 않음. 그래서 ``공분산 --(normalize)--> 상관계수``
- 공분산도 모공분산, 표본공분산으로 메소드가 나뉘므로 사용상황에 따라 잘 사용해야 함

In [17]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

df.select(corr("Quantity", "UnitPrice"), covar_pop("Quantity", "UnitPrice"), covar_samp("Quantity", "UnitPrice")).show()

+-------------------------+------------------------------+-------------------------------+
|corr(Quantity, UnitPrice)|covar_pop(Quantity, UnitPrice)|covar_samp(Quantity, UnitPrice)|
+-------------------------+------------------------------+-------------------------------+
|     -0.00123492454487...|           -26.058713170967906|            -26.058761257936858|
+-------------------------+------------------------------+-------------------------------+



### 복합 데이터 타입의 집계
- 예를들어, 특정 칼럼의 값을 리스트, 셋(set)으로 수집이 가능한 메소드들
- ``agg``메소드를 활용

In [20]:
from pyspark.sql.functions import collect_list, collect_set

df.agg(collect_list('Country'), collect_set('Country')).show(truncate=True)

+---------------------+--------------------+
|collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
| [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+



## 그룹화
- 데이터 그룹 기반의 집계를 수행
    - ex) 고유한 송장번호를 기준으로 그룹화 -> 그룹별 물품 수를 카운트
    - pandas ex) ``df.groupby('InvoiceNo')['물품'].count()``, ``df.groupby('InvoiceNo').agg({'물품': 'count})``
- 데이터 그룹화 집계 시에도 지연방식으로 수행됨
    - 데이터 그룹화 시, ``RelationalGroupedDataset``이 반환되고,
    - 집계 연산 수행 시, ``DataFrame``이 반환됨

In [21]:
# 그룹화시킬 칼럼을 다수 입력 가능 -> count를 메소드로 사용하는 방식
df.groupby("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   536596|      null|    6|
|   537252|      null|    1|
|   538041|      null|    1|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
+---------+----------+-----+
only showing top 20 rows



### 표현식을 이용한 그룹화
- ``count``는 메서드로도 사용이 가능하지만 import 해서 함수를 사용하는 것을 권고
- ``select``구문에서 집계를 사용하는 것보다 ``agg`` 메서드를 사용하는 것이 한 번에 여러 집계를 처리할 수 있으며 집계에 표현식(``expr``)을 사용할 수 있음

In [23]:
from pyspark.sql.functions import count # import해서 count를 메소드로 사용 

df.groupBy("InvoiceNo") \
  .agg(count("Quantity").alias('quan'),
      expr("count(Quantity) as quan2")
      ) \
  .show(10)

+---------+----+-----+
|InvoiceNo|quan|quan2|
+---------+----+-----+
|   536596|   6|    6|
|   536938|  14|   14|
|   537252|   1|    1|
|   537691|  20|   20|
|   538041|   1|    1|
|   538184|  26|   26|
|   538517|  53|   53|
|   538879|  19|   19|
|   539275|   6|    6|
|   539630|  12|   12|
+---------+----+-----+
only showing top 10 rows



## 윈도우 함수
- **윈도우 명세(Window Specification)**를 정의해주어야 함
- ``group-by``는 모든 row 레코드들이 단일 그룹으로만 이동
- 반면, 윈도우 함수는 프레임에 입력되는 모든 로우에 대해 결괏값을 계산
    - 프레임: row 그룹 기반의 테이블
    - 각 로우는 하나 이상의 프레임에 할당될 수 있음

In [64]:
# 날짜를 기준으로 윈도우 함수를 만들려 하는 예제이므로 date 칼럼 생성
from pyspark.sql.functions import col, to_date

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
# 임시 view 테이블로 생성 for 트랜스포메이션 적용 결과 보기 위함
dfWithDate.createOrReplaceTempView("dfWithDate")

- SQL의 윈도우 함수: ``window_function() over (partition by 칼럼1, 칼럼2 order by 칼럼3 desc)``
- Pyspark의 윈도우 함수
    - ``partitionBy``: coalesce 메소드와 같이 파티셔닝관련 메소드와는 전혀 관련 없음! 그룹을 어떻게 나눌지 결정하는 개념임!
    - ``orderBy``: ``partitionBy``의 정렬 방식을 의미
    - ``rowsBetween``: 윈도우 함수의 프레임 명세를 의미하며 입력된 row들의 참조를 기반으로 프레임에 row가 포함될 수 있는지 결정

In [65]:
# ex1 -> "고객, 시간대 별로 최대 구매 개수 집계"
# 첫 로우부터 현재로우까지 모두 프레임에 포함될 수 있는지 확인하는 명세서 예제임
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

# 윈도우 명세서
windowSpec = Window.partitionBy("CustomerId", "date")\
                   .orderBy(desc("Quantity"))\
                   .rowsBetween(Window.unboundedPreceding, Window.currentRow)


from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)  # over에 윈도우명세서를 입력

In [66]:
# 칼럼(표현식) 타입을 반환함 -> DataFrame의 select 구문에서 사용 가능 함!
print(type(maxPurchaseQuantity))

<class 'pyspark.sql.column.Column'>


In [67]:
# ex2 -> 고객별로 최대 구매 수량을 가진 날짜가 언제인지
from pyspark.sql.functions import rank, dense_rank

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [ ]:
# 이제 위에서 수행한 3개의 윈도우 함수 집계 값을 select in dataframe 하기 -> 날짜 파싱에러 발생(일, 시간 한자리, 두자리 포맷 불일치.. 이거 통일 시킬 순 없나..?)
from pyspark.sql.functions import col

dfWithDate.where("CustomerId IS NOT NULL")\
          .orderBy("CustomerId")\
          .select(col("CustomerId"),
                 col("date"),
                 col("Quantity"),
                 purchaseDenseRank.alias("quantityDenseRank"),
                 purchaseRank.alias("quantityRank"),
                 maxPurchaseQuantity.alias("maxPurchaseQuantity"))\
          .show()